# Parameter Searching

``Parameter searching`` is one of the most common things in computational modeling. When creating a 
model, we'll be presented with many parameters to control how our defined model evolves. Often times, 
we don't immediately know what the optimal parameter set should be for a given model, and thus we'd 
like to be able to explore a range of possibilities. 

Here, ``BrainPy`` provides a convinient and powerful way for parameter searching.

In [ ]:
import brainpy as bp
import numpy as np

In [ ]:
N = 5000
Vr = 10  # mV
theta = 20  # mV
tau = 20  # ms
delta = 2  # ms
taurefr = 2  # ms
duration = 100. # second
C = 1000
sparseness = float(C)/N
J = .1  # mV
muext = 25  # mV
sigmaext = 1  # mV


@bp.integrate
def int_v(v, t):
    return (-v + muext) / tau, sigmaext * np.sqrt(tau) / tau

def update(ST, _t):
    ST['spike'] = 0.
    if (_t - ST['t_last_spike']) > taurefr:
        v = int_v(ST['v'], _t)
        if v > theta:
            ST['spike'] = 1.
            ST['v'] = Vr
            ST['t_last_spike'] = _t
        else:
            ST['v'] = v
    

lif = bp.Ney


eqs = """
dV/dt = (-V+muext + sigmaext * sqrt(tau) * xi)/tau : volt
"""

group = NeuronGroup(N, eqs, threshold='V>theta',
                    reset='V=Vr', refractory=taurefr, method='euler')
group.V = Vr
conn = Synapses(group, group, on_pre='V += -J', delay=delta)
conn.connect(p=sparseness)
M = SpikeMonitor(group)
LFP = PopulationRateMonitor(group)

run(duration)

subplot(211)
plot(M.t/ms, M.i, '.')
xlim(0, duration/ms)

subplot(212)
plot(LFP.t/ms, LFP.smooth_rate(window='flat', width=0.5*ms)/Hz)
xlim(0, duration/ms)

show()